# 股票交易数据分析

In [1]:
%pwd

u'/Users/kamidox/work/stock-data'

In [2]:
import pandas as pd
import numpy as np

## 原始数据: 2000 年 - 2009 年 5 分钟历史成交数据

需要确保原始数据放在 raw 目录下，且每个年份单独一个目录。搜索 ghancn 可以免费下载 2009 年之前的数据。2009 年之后的需要购买。

目前的免费数据质量不高，里面有不少错误。具体参阅 README.md。

In [3]:
%ls raw

2000/ 2001/ 2002/ 2003/ 2004/ 2005/ 2006/ 2007/ 2008/


In [224]:
names = ['date',
         'time',
         'opening_price',
         'ceiling_price',
         'floor_price',
         'closing_price',
         'volume',
         'amount']
# 读取数据时，我们以日期为索引，并解析成日期格式
raw = pd.read_csv('raw/2008/SH600690.csv', names=names, header=None, index_col='date', parse_dates=True)
raw.head()

,time,opening_price,ceiling_price,floor_price,closing_price,volume,amount
date,,,,,,,
2008-01-02,09:35,22.50,22.63,22.50,22.51,2042.50,4604723
2008-01-02,09:40,22.51,22.51,22.29,22.37,1545.17,3460503
2008-01-02,09:45,22.39,22.62,22.38,22.62,1744.76,3921443
2008-01-02,09:50,22.60,23.00,22.60,22.95,5339.00,12225939
2008-01-02,09:55,22.98,23.20,22.89,23.20,12577.73,28947824


### 转化为日交易数据

In [225]:
# 股票涨跌幅检查，不能超过 10% ，过滤掉一些不合法的数据
def _valid_price(g):
    return (((g.max() - g.min()) / g.min()) < 0.223).all()

# 按照日期分组
days = raw.groupby(level=0).agg(
    {'opening_price': lambda g: _valid_price(g) and g[0] or 0,
     'ceiling_price': lambda g: _valid_price(g) and np.max(g) or 0,
     'floor_price': lambda g: _valid_price(g) and np.min(g) or 0,
     'closing_price': lambda g: _valid_price(g) and g[-1] or 0,
     'volume': 'sum',
     'amount': 'sum'})
days.head()

,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2008-01-02,22.29,22.50,24.50,200809.34,476179680,24.03
2008-01-03,23.81,24.03,25.20,166037.98,406906304,24.54
2008-01-04,23.68,24.53,24.76,149078.64,358418560,24.17
2008-01-07,23.75,24.03,24.75,93950.43,227289136,24.38
2008-01-08,23.49,24.38,24.38,149056.24,355752416,23.53


### 合并数据

可以参阅 stock.py 里的 `main()` 函数。把所有的数据转化为日交易数据，然后以股票代号为文件名保存在 `data` 目录下。

## 选股

什么股票是好股票？要回答这个问题，先要把最简单的问题说清楚。炒股就是低买高卖，实现获利。那么好股票的标准就是在你的持股周期内，**波动最大的股票**。这很好理解吧，波动最大，我们才有可能在相对低点买入，在相对高点卖出，获利最大。

在一定的时间周期内，**衡量股票波动的指标定义为 最高价/最低价**。以我们表格中的数据，就是 ceiling_price/floor_price。这个比率最大的股票就是好股票。

关于时间周期，这个和炒股策略有关。有些人喜欢做短线，可能就持股几天，或一两周。有些人习惯做长线，可能持股几个月甚至几年。

有了这个思路，我们就可以玩转已经转换为日交易数据的股票，选出近期波动最大的股票。假设我们的目标是**选出一个月内波动最大的股票**。我们看一下如何用 pandas 实现这个目标。

### 过滤数据

我们先要按照考查周期来过滤数据。为了简单起见，我们假设一个月是20个交易日，且只计算交易时间，不计算停牌时间。

这里，我们直接使用青岛海尔 600690 这个股票来作为示例。我们直接读取已经合并过的数据。

In [6]:
qdhr = pd.read_csv('data/SH600690.csv')
qdhr.head()

,date,floor_price,opening_price,ceiling_price,volume,amount,closing_price
0,2000/01/04,16.21,16.31,17.30,17050,28651788,17.08
1,2000/01/05,16.71,17.10,17.39,17486,29893382,16.75
2,2000/01/06,16.62,16.72,18.45,41268,74191124,18.45
3,2000/01/07,18.45,18.45,20.30,187321,379826144,20.30
4,2000/01/10,20.30,20.30,22.33,317307,695940608,21.65


In [7]:
len(qdhr)

2111

In [8]:
import numpy as np

# 定义产生分组索引的函数，比如我们要计算的周期是 20 天，则按照日期，20 个交易日一组
def gen_item_group_index(total, group_len):
    """ generate an item group index array 
    
    suppose total = 10, unitlen = 2, then we will return array [0 0 1 1 2 2 3 3 4 4]
    """
    
    group_count = total / group_len
    group_index = np.arange(total)
    for i in range(group_count):
        group_index[i * group_len: (i + 1) * group_len] = i
    group_index[(i + 1) * group_len : total] = i + 1
    return group_index.tolist()

gen_item_group_index(10, 3)

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3]

In [9]:
group_len = 20

group_index = gen_item_group_index(len(qdhr), group_len)
# 把分组索引数据添加到股票数据里
qdhr['group_index'] = group_index
qdhr

,date,floor_price,opening_price,ceiling_price,volume,amount,closing_price,group_index
0,2000/01/04,16.21,16.31,17.30,17050.00,28651788,17.08,0
1,2000/01/05,16.71,17.10,17.39,17486.00,29893382,16.75,0
2,2000/01/06,16.62,16.72,18.45,41268.00,74191124,18.45,0
3,2000/01/07,18.45,18.45,20.30,187321.00,379826144,20.30,0
4,2000/01/10,20.30,20.30,22.33,317307.00,695940608,21.65,0
5,2000/01/11,20.18,21.70,21.70,108273.00,225728608,20.36,0
6,2000/01/12,19.52,20.20,20.66,120790.00,241688948,19.77,0
7,2000/01/13,18.66,19.80,19.99,63430.00,122255604,18.75,0
8,2000/01/14,18.28,18.80,19.10,40871.00,76201248,18.34,0
9,2000/01/17,18.08,18.36,19.05,30272.00,56388926,18.80,0


In [10]:
# 根据索引分组计算
qdhr_group = qdhr.groupby('group_index').agg({'date': 'first', 'volume': 'sum', 'floor_price': 'min', 'ceiling_price': 'max'})
qdhr_group.head()

,date,volume,ceiling_price,floor_price
group_index,,,,
0,2000/01/04,1371665.00,22.33,16.21
1,2000/02/15,2996811.02,27.20,19.00
2,2000/03/14,905659.24,24.10,20.08
3,2000/04/11,414194.92,23.49,20.10
4,2000/05/16,295170.08,23.36,20.15


In [11]:
# 添加我们的波动指标 股票波动系数 = 最高价/最低价
qdhr_group['ripples_radio'] = qdhr_group.ceiling_price / qdhr_group.floor_price
qdhr_group.head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
0,2000/01/04,1371665.00,22.33,16.21,1.377545
1,2000/02/15,2996811.02,27.20,19.00,1.431579
2,2000/03/14,905659.24,24.10,20.08,1.200199
3,2000/04/11,414194.92,23.49,20.10,1.168657
4,2000/05/16,295170.08,23.36,20.15,1.159305


In [12]:
# 降序排列。我们把分组的起始日期，交易量总和都列出来，也可以观察一下交易量和股票波动比的关系
qdhr_ripples = qdhr_group.sort_values('ripples_radio', ascending=False)
qdhr_ripples.head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
81,2006/12/20,4357087.54,11.37,6.90,1.647826
98,2008/05/21,1388115.89,13.47,8.21,1.640682
53,2004/06/30,223884.55,8.96,5.50,1.629091
95,2008/02/21,1542367.18,23.23,14.34,1.619944
97,2008/04/18,4551911.34,14.85,9.18,1.617647


In [13]:
# 国内股票市场没有做空机制，所以如果波动的股票是先出现高价格，再出现低价格，这样的波动我们应该过滤掉。
# 我们应该只关注一个分组周期内，先出现低价，再出现高价的波动，即上涨趋势的波动。这些才是有效波动。
# 另外一个我们过滤掉下跌波动的原因是，股票经常会进行除权。而目前我们又没有足够的数据去处理除权。

def is_valid_ripples(group):
    """ filter out invalid ripples """
    id_of_floor_price = group['floor_price'].idxmin()
    id_of_ceiling_price = group['floor_price'].idxmax()
    return id_of_ceiling_price > id_of_floor_price

qdhr_valid = qdhr.groupby('group_index').filter(is_valid_ripples)
# 从过滤的结果可以看出来，原来有 2111 行数据，过滤完只剩下 1000 行了
qdhr_valid

,date,floor_price,opening_price,ceiling_price,volume,amount,closing_price,group_index
0,2000/01/04,16.21,16.31,17.30,17050.00,28651788,17.08,0
1,2000/01/05,16.71,17.10,17.39,17486.00,29893382,16.75,0
2,2000/01/06,16.62,16.72,18.45,41268.00,74191124,18.45,0
3,2000/01/07,18.45,18.45,20.30,187321.00,379826144,20.30,0
4,2000/01/10,20.30,20.30,22.33,317307.00,695940608,21.65,0
5,2000/01/11,20.18,21.70,21.70,108273.00,225728608,20.36,0
6,2000/01/12,19.52,20.20,20.66,120790.00,241688948,19.77,0
7,2000/01/13,18.66,19.80,19.99,63430.00,122255604,18.75,0
8,2000/01/14,18.28,18.80,19.10,40871.00,76201248,18.34,0
9,2000/01/17,18.08,18.36,19.05,30272.00,56388926,18.80,0


In [14]:
# 针对有效的数据，按日期分组，计算波动，排序。可以看到有效波动，即最大的上涨的波动的 5 个数据
qdhr_group_valid = qdhr_valid.groupby('group_index').agg({'date': 'first', 'volume': 'sum', 'floor_price': 'min', 'ceiling_price': 'max'})
qdhr_group_valid['ripples_radio'] = qdhr_group_valid.ceiling_price / qdhr_group_valid.floor_price
qdhr_ripples_valid = qdhr_group_valid.sort_values('ripples_radio', ascending=False)
qdhr_ripples_valid.head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
81,2006/12/20,4357087.54,11.37,6.90,1.647826
97,2008/04/18,4551911.34,14.85,9.18,1.617647
86,2007/05/28,5703121.25,18.89,11.85,1.594093
1,2000/02/15,2996811.02,27.20,19.00,1.431579
0,2000/01/04,1371665.00,22.33,16.21,1.377545


In [15]:
# 把所有的波动和上面有效的波动数据对比
qdhr_ripples.head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
81,2006/12/20,4357087.54,11.37,6.90,1.647826
98,2008/05/21,1388115.89,13.47,8.21,1.640682
53,2004/06/30,223884.55,8.96,5.50,1.629091
95,2008/02/21,1542367.18,23.23,14.34,1.619944
97,2008/04/18,4551911.34,14.85,9.18,1.617647


In [16]:
# 我们算出前 10 个有效波动，即上涨波动的平均值。作为这个股票的波动值。
# 最后，我们就可以根据所有股票的波动值来选择最优的股票了。
qdhr_ripples_valid.head(10).ripples_radio.mean()

1.4261320425918829

## 使用 stock.py 来过滤数据

In [17]:
import stock as st

In [226]:
ripples = st.get_stock_ripples('data/SZ000564.csv', period=20)

mean ripples range on top 10 in period of 20 for data/SZ000564.csv: 7.9405


In [243]:
# 显示指定周期内，股票的前十个波动周期
ripples.head(10)

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
22,2001/11/12,118590.77,503.38,7.50,67.117333
85,2007/03/27,3949438.83,8.50,5.82,1.460481
83,2007/01/23,2205300.93,5.36,3.69,1.452575
74,2006/04/26,1832939.11,3.40,2.35,1.446809
1,2000/02/15,1437101.24,11.68,8.60,1.358140
86,2007/04/25,4444983.04,10.60,7.95,1.333333
82,2006/12/21,2693370.92,4.35,3.28,1.326220
84,2007/02/27,2468582.33,6.10,4.60,1.326087
24,2002/01/11,176207.17,7.30,5.65,1.292035


In [255]:
# 有时候，我们想看一下某个股票某个波动周期下的原始数据
reload(st)
stock_file = 'test-data/SH600691.csv'
st.ripple_raw_data(stock_file, ripple_idx=0, days=30)

mean ripples range on top 10 in period of 20 for test-data/SH600691.csv: 1.6287


,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2007-01-12,4.25,4.28,4.39,7546.14,3265550,4.35
2007-01-15,4.30,4.35,4.43,8827.59,3857417,4.41
2007-01-16,4.40,4.40,4.63,8975.07,4072200,4.63
2007-01-17,4.61,4.61,4.86,12056.67,5832008,4.86
2007-01-18,4.81,4.91,5.09,20179.03,9933153,4.99


In [218]:
def _valid_price(g):
    return (((g.max() - g.min()) / g.min()) < 0.223).all()

l = [0.9, 1.1, 0.9, 1.1, 0.9, 1.1]
_valid_price(pd.Series(l))

True

In [180]:
names = ['date',
         'time',
         'opening_price',
         'ceiling_price',
         'floor_price',
         'closing_price',
         'volume',
         'amount']
data = pd.read_csv('test-raw/2008/SZ161607.csv', names=names, header=None, index_col='date', parse_dates=True)
data.head(48)

,time,opening_price,ceiling_price,floor_price,closing_price,volume,amount
date,,,,,,,
2008-01-02,09:35,1.789,1.794,1.780,1.780,27480,49185.32
2008-01-02,09:40,1.780,1.780,1.775,1.779,23600,41927.40
2008-01-02,09:45,1.779,1.779,1.770,1.770,31900,56622.00
2008-01-02,09:50,1.770,1.770,1.757,1.757,60500,106457.70
2008-01-02,09:55,1.757,1.770,1.757,1.770,47000,82682.98
2008-01-02,10:00,1.760,1.771,1.760,1.771,60472,107038.91
2008-01-02,10:05,1.771,1.771,1.771,1.771,18424,32628.91
2008-01-02,10:10,1.771,1.774,1.771,1.774,9700,17207.78
2008-01-02,10:15,1.774,1.775,1.774,1.775,46000,81624.63


In [230]:
# 股票涨跌幅检查，不能超过 10%
def _valid_price(g):
    return (((g.max() - g.min()) / g.min()) < 0.223).all()

days = data.groupby(level=0).agg(
                                {'opening_price': lambda g: _valid_price(g) and g[0] or 0,
                                 'ceiling_price': lambda g: _valid_price(g) and np.max(g) or 0,
                                 'floor_price': lambda g: _valid_price(g) and np.min(g) or 0,
                                 'closing_price': lambda g: _valid_price(g) and g[-1] or 0,
                                 'volume': 'sum',
                                 'amount': 'sum'})
days.loc['2008-11-19']

floor_price      0.000000e+00
opening_price    0.000000e+00
ceiling_price    0.000000e+00
volume           5.939168e+09
amount           6.021043e+32
closing_price    0.000000e+00
Name: 2008-11-19 00:00:00, dtype: float64